In [8]:
class Entry(object):
    
    def __init__(self, entry_signal, stop_loss=1.00, profit_taker=1.00,
                 entry_price_offset=-0.25, trade_wait_tm=4, quantity=1,
                 action='BUY', pr_per_point=5.0, **kwargs):
        self.entry_signal = entry_signal
        self.stop_loss = stop_loss
        self.profit_taker = profit_taker
        self.entry_price_offset = entry_price_offset
        self.trade_wait_tm = trade_wait_tm
        self.quantity = quantity
        self.action = action
        self.sign =  1 if self.action == 'BUY' else -1
        self.wins = 0
        self.losses = 0
        self.entry_tms = []
        self.exit_tms = []
        self.profits = []
        self.cum_profits = []
        self.cum_profit = 0
        self.pr_per_point = pr_per_point
        self.in_position = False
        self.kwargs = kwargs
    
    def enter(self, online_indicator):
        return self.entry_signal(online_indicator, **self.kwargs)
    
    def entry(self, entry_tm):
        self.in_position = True
        self.entry_tms.append(entry_tm)
    
    def exit(self, exit_tm, win_loss):
        self.in_position = False
        self.exit_tms.append(exit_tm)
        self.profits.append(win_loss * self.pr_per_point)
        if win_loss == 1:
            self.wins +=1
        else:
            self.losses += 1
        if len(self.cum_profits) == 0:
            self.cum_profits.append(self.profits[-1])
        else:
            self.cum_profits.append(self.cum_profits[-1] + self.profits[-1])
        self.cum_profit = self.cum_profits[-1]

def scalping_trend(online_indicator, rsi_threshold=70, pr_diff_ewa_thresh=0, adx_threshold=15):
    indicators =  online_indicator.temp_indicator
    condition = indicators.rsi < rsi_threshold
    condition = condition and indicators.pr_diff_ewa > pr_diff_ewa_thresh
    condition = condition and indicators.mean_pr > indicators.sma_pr 
    condition = condition and indicators.adx > adx_threshold
    condition = condition and indicators.plus_dir_ind > indicators.min_dir_ind
    return condition

def rsi_long(online_indicator, rsi_threshold=30, pr_diff_ewa_thresh=0, adx_threshold=15):
    indicators =  online_indicator.temp_indicator
    condition = indicators.rsi < rsi_threshold and indicators.adx > 15 and indicators.mean_pr > indicators.sma_pr
    return condition

def rsi_short(online_indicator, rsi_threshold=70, pr_diff_ewa_thresh=0, adx_threshold=15):
    indicators =  online_indicator.temp_indicator
    condition = indicators.rsi > rsi_threshold and indicators.adx > 15 and indicators.mean_pr < indicators.sma_pr
    return condition

In [9]:
import pickle

entry_objs = pickle.load(open('tracked_runs/run1.pickle', 'rb'))

In [18]:
strategy_cum_profits = {}
strategy_entry_tms = {}
straegy_exit_tms = {}

for strategy in entry_objs:
    strategy_cum_profits[strategy] = entry_objs[strategy].cum_profits
    strategy_entry_tms[strategy] = entry_objs[strategy].entry_tms
    straegy_exit_tms[strategy] = entry_objs[strategy].exit_tms

In [19]:
import plotly.graph_objects as go

fig = go.Figure()

for strategy in entry_objs:
    fig.add_trace(
        go.Scattergl(x=straegy_exit_tms[strategy], y=strategy_cum_profits[strategy], mode='lines+markers',
        name=strategy)
    )

fig